## В этом упражнении вы будете работать с единым журналом пользователей, посетивших страницу, включая их временные метки.

-------------------------------------------------------------------------------------------------------------------------------

### 1. Создать представления dataframe с двумя столбцами: datetime и user, прочитав feed-views.log
- преобразовать дату и время в тип datetime64[ns]
- извлечь год, месяц, день, час, минуту и секунду из значений этого столбца в новые столбцы

In [1]:
import pandas as pd

df = pd.read_csv('../data/feed-views.log', sep='\t', names=['datetime', 'user'])

df['datetime'] = pd.to_datetime(df['datetime'])

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second

print(df)

                       datetime       user  year  month  day  hour  minute  \
0    2020-04-17 12:01:08.463179      artem  2020      4   17    12       1   
1    2020-04-17 12:01:23.743946      artem  2020      4   17    12       1   
2    2020-04-17 12:27:30.646665      artem  2020      4   17    12      27   
3    2020-04-17 12:35:44.884757      artem  2020      4   17    12      35   
4    2020-04-17 12:35:52.735016      artem  2020      4   17    12      35   
...                         ...        ...   ...    ...  ...   ...     ...   
1071 2020-05-21 18:45:20.441142  valentina  2020      5   21    18      45   
1072 2020-05-21 23:03:06.457819      maxim  2020      5   21    23       3   
1073 2020-05-21 23:23:49.995349      pavel  2020      5   21    23      23   
1074 2020-05-21 23:49:22.386789      artem  2020      5   21    23      49   
1075 2020-05-22 10:36:14.662600      artem  2020      5   22    10      36   

      second  
0          8  
1         23  
2         30  
3  

### 2. Cоздать новую колонку дневное время
- вам необходимо присвоить конкретное значение времени суток, если час находится в определенном интервале, например, полдень, если час больше 11 и меньше или равен 17
- 0 – 3.59 ночь, 4 – 6.59 раннее утро, 7 – 10.59 утро, 11 – 16.59 день, 17 – 19.59 ранний вечер, 20 – 23.59 вечер
- используйте метод cut для решения этой подзадачи
- назначить столбец user в качестве индекса

In [2]:
bins = [0, 4, 7, 11, 17, 20, 24]
labels = ['night', 'early morning', 'morning', 'afternoon', 'early evening', 'evening']
df['daytime'] = pd.cut(df['hour'], bins=bins, labels=labels, right=True)

df.set_index('user', inplace=True)

print(df)


                            datetime  year  month  day  hour  minute  second  \
user                                                                           
artem     2020-04-17 12:01:08.463179  2020      4   17    12       1       8   
artem     2020-04-17 12:01:23.743946  2020      4   17    12       1      23   
artem     2020-04-17 12:27:30.646665  2020      4   17    12      27      30   
artem     2020-04-17 12:35:44.884757  2020      4   17    12      35      44   
artem     2020-04-17 12:35:52.735016  2020      4   17    12      35      52   
...                              ...   ...    ...  ...   ...     ...     ...   
valentina 2020-05-21 18:45:20.441142  2020      5   21    18      45      20   
maxim     2020-05-21 23:03:06.457819  2020      5   21    23       3       6   
pavel     2020-05-21 23:23:49.995349  2020      5   21    23      23      49   
artem     2020-05-21 23:49:22.386789  2020      5   21    23      49      22   
artem     2020-05-22 10:36:14.662600  20

### 3. Подсчитать количество элементов в вашем фрейме данных
- используйте метод count()
- подсчитать количество элементов в каждой категории времени суток, используя метод value_counts()

In [3]:
print(df.size) # считаем элементы
print(df['daytime'].value_counts()) # в каждой категории времени суток

8608
daytime
evening          445
afternoon        279
early evening    144
morning           74
night             69
early morning      0
Name: count, dtype: int64


### 4. Сортировать значения в вашем фрейме данных по часам, минутам и секундам в порядке возрастания (одновременно, а не по одному)

In [4]:
print(df.sort_values(by=['hour', 'minute', 'second']))

                            datetime  year  month  day  hour  minute  second  \
user                                                                           
valentina 2020-05-15 00:00:13.222265  2020      5   15     0       0      13   
valentina 2020-05-15 00:01:05.153738  2020      5   15     0       1       5   
pavel     2020-05-12 00:01:27.764025  2020      5   12     0       1      27   
pavel     2020-05-12 00:01:38.444917  2020      5   12     0       1      38   
pavel     2020-05-12 00:01:55.395042  2020      5   12     0       1      55   
...                              ...   ...    ...  ...   ...     ...     ...   
artem     2020-05-21 23:49:22.386789  2020      5   21    23      49      22   
anatoliy  2020-05-09 23:53:55.599821  2020      5    9    23      53      55   
pavel     2020-05-09 23:54:54.260791  2020      5    9    23      54      54   
valentina 2020-05-14 23:58:56.754866  2020      5   14    23      58      56   
alexander 2020-05-14 23:59:38.758438  20

### 5. Рассчитать минимум и максимум для часов и режим для дневных категорий
- вычислить максимум часа для строк, где время суток — ночь
- вычислить минимум часа для строк, где время суток — утро
- В дополнение к этому, выясните, кто посещал страницу в эти часы (приведите один пример из этого)
- рассчитать мод на час и день

In [5]:
max_hour = df[df['daytime'] == 'night']['hour'].max()
min_hour = df[df['daytime'] == 'morning']['hour'].min()
print(f"Максимальный час ночью: {max_hour}, Пример юзера: {df[(df['daytime'] == 'night') & (df['hour'] == max_hour)].index.tolist()[0]}")
print(f"Минимальный час утром: {min_hour}, Пример юзера: {df[(df['daytime'] == 'morning') & (df['hour'] == min_hour)].index.tolist()[0]}")

print(f"Наиболее часто встречающийся час: {df['hour'].mode()[0]}")
print(f"Наиболее часто встречающееся время суток: {df['daytime'].mode()[0]}")

Максимальный час ночью: 4, Пример юзера: konstantin
Минимальный час утром: 8, Пример юзера: alexander
Наиболее часто встречающийся час: 22
Наиболее часто встречающееся время суток: evening


### 6. Показать 3 самых ранних часа утра и соответствующие им имена пользователей, а также 3 самых поздних часа и имена пользователей с помощью nsmallest() и nlargest()

In [6]:
print(f"Три самых ранних часа утра и юзеры: \n\n{df[df['daytime'] == 'morning'].nsmallest(3, 'hour')}\n\n\n")
print(f"Три самых поздних часа и юзеры: \n\n{df.nlargest(3, 'hour')}")

Три самых ранних часа утра и юзеры: 

                            datetime  year  month  day  hour  minute  second  \
user                                                                           
alexander 2020-05-15 08:16:03.918402  2020      5   15     8      16       3   
alexander 2020-05-15 08:35:01.471463  2020      5   15     8      35       1   
artem     2020-04-24 09:42:47.598208  2020      4   24     9      42      47   

           daytime  
user                
alexander  morning  
alexander  morning  
artem      morning  



Три самых поздних часа и юзеры: 

                             datetime  year  month  day  hour  minute  second  \
user                                                                            
konstantin 2020-04-18 23:06:34.198911  2020      4   18    23       6      34   
artem      2020-04-18 23:40:32.666884  2020      4   18    23      40      32   
artem      2020-04-19 23:10:03.761243  2020      4   19    23      10       3   

            d

### 7. Использовать метод describe() для получения базовой статистики по столбцам
- чтобы узнать, какой интервал посещения страницы является наиболее популярным, рассчитайте межквартильный размах для часа, извлекая значения из результата метода describe() и сохраняя его в переменной iqr

In [7]:
description = df.describe()
iqr = description.loc['75%', 'hour'] - description.loc['25%', 'hour']
# Третий квартиль — это значение, ниже которого находится 75% данных, Первый квартиль — это значение, ниже которого находится 25% данных.
# Межквартильный размах (IQR) — это разница между третьим и первым квартилями. 
# Он используется как мера статистической дисперсии и показывает диапазон, в котором находится 50% центральных данных.
# Настолько разбросаны значения часов в наборе данных:
print(iqr)

9.0


In [ ]:
views.hour_mod